<a href="https://colab.research.google.com/github/dingzhang2023/problem-solving-practice/blob/colab/Trie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Trie

**04/07/2024**

#### Core idea

> -  Group the string according to their prefixes, where strings with the same first letter are grouped together, and so on.
> -  To ensure that each letter corresponds to a specific node in a tree where each node's parent is the previous node's corresponding letter, `s[i]` is the parent node of `s[i + 1]`.
> - Example of trie, a trie containing the words `apple`, `app`, `banana`, `bat` and `cat`. Here's how the Trie looks:

                  (root)
                  /
                a
                / \
              p   b
              / \   \
            p   p   a
            /     \   \
          l       l   t
          /         \
        p           e
        /
        p
        |
        e

> - Save extra information in each node based on problems, such as `is_word` and `cnt`.


#### Major use cases
The main problems solved by Trie include:

1. String problems related to prefix
2. XOR Number problems, which can be converted to binary representation using trie


#### Trie code template
[208. Implement Trie (Prefix Tree)](https://leetcode.com/problems/implement-trie-prefix-tree/description/)
> Trie template problem
> - Insert
> - Query
> - Extra information saved in each node of trie, for this problem, flag `is_word` saved for id this node is a word or not.

In [ ]:
class Node:
    __slot__ = 'son', 'is_word'


    def __init__(self):
        # use array
        # self.son = [None] * 26
        # use defaultdice(Node)
        self.son = defaultdict(Node)
        self.is_word = False


class Trie:

    def __init__(self):
        self.root = Node()


    def insert(self, word: str) -> None:
        cur = self.root
        for c in word:
            cur = cur.son[c]
        cur.is_word = True

    def search(self, word: str) -> bool:
        cur = self.root
        for c in word:
            if c not in cur.son:
                return False
            cur = cur.son[c]
        return cur.is_word

    def startsWith(self, prefix: str) -> bool:
        cur = self.root
        for c in prefix:
            if c not in cur.son:
                return False
            cur = cur.son[c]
        return True

#### Problem lists

#### Trie code template
[1804. Implement Trie (Prefix Tree II)](https://leetcode.com/problems/implement-trie-ii-prefix-tree/description/)
> Trie template problem
> - Insert
> - Query
> - Erase, use lazy delete technique, use `word_cnt` information to each node to check the current node is the end of a word.

In [ ]:
class Node:
    __slots__ = 'son', 'prefix_cnt', 'word_cnt'

    def __init__(self):
        self.son = defaultdict(Node)
        self.prefix_cnt = 0
        self.word_cnt = 0

class Trie:

    def __init__(self):
        self.root = Node()


    def insert(self, word: str) -> None:
        cur = self.root
        for c in word:
            cur = cur.son[c]
            cur.prefix_cnt += 1
        cur.word_cnt += 1


    def countWordsEqualTo(self, word: str) -> int:
        cur = self.root
        for c in word:
            if c not in cur.son:
                return 0
            cur = cur.son[c]
        return cur.word_cnt

    def countWordsStartingWith(self, prefix: str) -> int:
        cur = self.root
        for c in prefix:
            if c not in cur.son:
                return 0
            cur = cur.son[c]
        return cur.prefix_cnt

    def erase(self, word: str) -> None:
        cur = self.root
        for c in word:
            if c not in cur.son:
                return
            cur = cur.son[c]
            cur.prefix_cnt -= 1
        cur.word_cnt -= 1

[2416. Sum of Prefix Scores Strings](https://leetcode.com/problems/sum-of-prefix-scores-of-strings/)

> The essence of counting the number of strings passing through a node `x`, where `x` is also a prefix, can be effectively achieved using a variable to store the count of prefixes passing through each node. This application is indeed a classic use case of Trie data structure.

> Insert each word into a Trie tree, then use Depth-First Search (DFS) starting from the root node to calculate the cumulative score of each character node, which is the answer.

> Implementation tricks, using ids list to save the words pass this node.

In [ ]:
class Node:
    __slots__ = 'son', 'score', 'ids'

    def __init__(self):
        self.son = dict()
        # self.son = defaultdict(Node)
        self.score = 0
        self.ids = []


class Solution:
    def sumPrefixScores(self, words: List[str]) -> List[int]:

        root = Node()
        # Insert word to trie
        for i, word in enumerate(words):
            cur = root
            for c in word:
                # comment if condition if use defaultdict()
                if c not in cur.son:
                    cur.son[c] = Node()
                cur = cur.son[c]
                cur.score += 1
            cur.ids.append(i) # save the word passing the path

        ans = [0] * len(words)
        def dfs(node, total_score):
            if node is None:
                return

            total_score += node.score
            for i in node.ids:
                ans[i] = total_score
            for child in node.son.values():
                dfs(child, total_score)

        dfs(root, 0)
        return ans

[3045. Count Prefix and Suffix Pairs II](https://leetcode.com/problems/count-prefix-and-suffix-pairs-ii/description/)

**Method I**
> To transform the problem of determining whether one string is a prefix of another into a problem of only checking for prefixes

> - Convert `s` to a list of pairs: [(s[0], s[n-1]), (s[1], s[n-2]),...,(s[n-1], s[0])]
> - Check whether the pair list associated with `words[i]` is a prefix of the pair list associated with `words[j]`
> - Use trie to check the prefix

In [ ]:
class Node:
  __slot__ = 'son', 'cnt'

  def __init__(self):
    self.son = dict() # key is pair, value is node
    self.cnt = 0 # the count of the words(pair) ends with this node

class Solution:
    def countPrefixSuffixPairs(self, words: List[str]) -> int:
        ans = 0
        root = Node()
        for s in words:
            cur = root
            for p in zip(s, s[::-1]):
                # p = (s[i], s[n-1-i])
                if p not in cur.son:
                    cur.son[p] = Node()
                cur = cur.son[p]
                ans += cur.cnt
            # update the count of pair ends with s
            cur.cnt += 1
        return ans

Method II
> If `s` is both a prefix and a suffix of `t`, then for `t`, the length `|S|` of its prefix and suffix must be the same. The Z-function is defined as z[i] = LCP(s[i:], s) = n - i
>


[3093. Longest Common Suffix Queries](https://leetcode.com/problems/longest-common-suffix-queries/)

> This problem only matches suffixes. After reversing the string, it becomes prefixes, which can be treated as multiple strings to solve the prefix matching problem. This is a typical use case of Trie.

> Insert string to trie then query.

> Implementation tricks, saving index and length of each word on the node of trie.

> Edge case, empty string.

Using list to implement node of trie

In [ ]:
class Node:
    __slots__ = 'son', 'min_len', 'i'

    def __init__(self):
        self.son = [None] * 26
        self.min_l = inf

class Solution:
    def stringIndices(self, wordsContainer: List[str], wordsQuery: List[str]) -> List[int]:
        ord_a = ord('a')
        root = Node()
        for idx, s in enumerate(wordsContainer):
            l = len(s)
            cur = root
            if l < cur.min_len:
                cur.min_len, cur.i = l, idx
            for c in map(ord, reversed(s)):
                c -= ord_a
                if cur.son[c] is None:
                    cur.son[c] = Node()
                cur = cur.son[c]
                # update the current node with the shortest prefix
                if l < cur.min_l:
                    cur.min_l, cur.i = l, idx

        ans = []
        for s in wordsQuery:
            cur = root
            for c in map(ord, reversed(s)):
                c -= ord_a
                if cur.son[c] is None:
                    break
                cur = cur.son[c]
            ans.append(cur.i)
        return ans

Using defaultdict to implement node of trie

In [ ]:
class Node:
    __slots__ = 'son', 'min_l', 'i'

    def __init__(self):
        self.son = defaultdict(Node)
        self.min_l = inf

class Solution:
    def stringIndices(self, wordsContainer: List[str], wordsQuery: List[str]) -> List[int]:
        root = Node()
        for idx, s in enumerate(wordsContainer):
            l = len(s)
            cur = root
            if l < cur.min_l:
                cur.min_l, cur.i = l, idx
            for c in reversed(s):
                cur = cur.son[c]
                if l < cur.min_l:
                    cur.min_l, cur.i = l, idx

        ans = []
        for s in wordsQuery:
            cur = root
            for c in reversed(s):
                # prefix does not exist
                if c not in cur.son:
                    break
                cur = cur.son[c]
            ans.append(cur.i)
        return ans

#### Trie application in `XOR` problem 0-1 Trie



#### Template problem
[421. Maximum XOR of Two Numbers in an Array](https://leetcode.com/problems/maximum-xor-of-two-numbers-in-an-array/description/)
> Think each number in binary, for each bit 0 we need to check if there is a number whose current bit is 1, a data structure needed to main 0 and 1 and group each bit together based on 0 or 1.

> Trie comes into play, for each bit from the left(highest one) to right, 0 or 1 can be seperated into left and right subtree, which is a smaller similar problem.

> Each node of trie can be reused, which optimizes the space.

> Important thinking is to use trie to **group** each bit for all numbers from the highest to lowest bit.

[2935.  Maximum Strong Pair XOR II](https://leetcode.com/problems/maximum-strong-pair-xor-ii/description/)

>